In [1]:
import pandas as pd
from sklearn import ensemble
#from sklearn.datasets import make_classification 
from deslib.dcs.ola import OLA 
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
from sklearn.ensemble import BaggingClassifier       
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier  
from sklearn.neural_network import MLPClassifier 
from math import sqrt
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
#----------------------------------------------------------------------------------------------------資料前處理
train_x = train[['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember','Exited' ]]

In [3]:
train_y = train_x['Exited'] 
train_x = train_x[['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember' ]]
Std = StandardScaler()
train_x[['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts' ]] = Std.fit_transform(train_x[['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts' ]]) #標準化
train_x[['HasCrCard','IsActiveMember']] = train_x[['HasCrCard','IsActiveMember']].astype(str)
train_x_dummies = pd.get_dummies(train_x)  
#------------------------------------------------------------------------------------------------------------------------------
test_x = test[['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember' ]]
test_x[['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts' ]] = Std.fit_transform(test_x[['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts' ]]) #標準化
test_x[['HasCrCard','IsActiveMember']] = test_x[['HasCrCard','IsActiveMember']].astype(str)
test_x_dummies = pd.get_dummies(test_x)
dummy = test_x_dummies.columns.tolist() 

<ipython-input-3-36dfc1c3be82>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_x[['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts' ]] = Std.fit_transform(test_x[['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts' ]]) #標準化
c:\python37\lib\site-packages\pandas\core\indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
c:\python37\lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying t

In [4]:
classifiers = [BaggingClassifier(n_estimators=80,max_samples=0.71, max_features=0.5),HistGradientBoostingClassifier(max_iter=80,learning_rate = 1, max_depth = 1)] 
# fit each classifier on the training set
for c in classifiers:
    c.fit(train_x_dummies, train_y)
# define the DCS-LA model
model = OLA(pool_classifiers=classifiers)
parameters = {'learning_rate': [0.01, 0.05, 0.1], 'max_depth': [2,3,4, 5, 6]}

clf = GridSearchCV(model, param_grid=parameters, scoring='roc_auc')
clf.fit(train_x_dummies, train_y)

pred_y =clf.predict(train_x_dummies)

ValueError: Invalid parameter learning_rate for estimator OLA(pool_classifiers=[BaggingClassifier(max_features=0.5, max_samples=0.71,
                                        n_estimators=80),
                      HistGradientBoostingClassifier(learning_rate=1,
                                                     max_depth=1,
                                                     max_iter=80)]). Check the list of available parameters with `estimator.get_params().keys()`.

In [ ]:
#classifiers = [BaggingClassifier(n_estimators=80,max_samples=0.71, max_features=0.5),HistGradientBoostingClassifier(max_iter=80,learning_rate = 1, max_depth = 1)] 
# fit each classifier on the training set
#for c in classifiers:
#    c.fit(train_x_dummies, train_y)
# define the DCS-LA model
#model = OLA(pool_classifiers=classifiers)
# fit the model
#model.fit(train_x_dummies, train_y) 
# make predictions on the test set
#pred_y = model.predict(test_x_dummies)  

In [ ]:
final = {"RowNumber":test["RowNumber"],'Exited':pred_y}
final_df = pd.DataFrame(final)
final_df.to_csv('final.csv')